<a href="https://colab.research.google.com/github/evanscastonguay/colab/blob/main/orphanbank_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![sslogo](https://github.com/stratascratch/stratascratch.github.io/raw/master/assets/sslogo.jpg)

# Web scraping in Python

Scraping refers to extracting useful data from web pages which are written in a programming language called HTML. To scrap data from the HTML tree we first have to download the web page to our PC.

### Install the packages using pip

In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

### Import the modules

In [ ]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

## Full Example

In [ ]:
def scrapePage ( searchName) :
  print ("scrapePages")
  # Get the column names of our dataframe. 
  columns = ["compte", "nom", "address", "montant", "beneficiare", "bank", "history"]
  rows = []

  for page in range (1,25):
    url = "https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page="+str(page)+"&searchType=Person&lastName="+str(searchName)+"&propertyId=0"
    print (url)
    web_page = bs4.BeautifulSoup(requests.get(url, headers={ "UserAgent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.183 Safari/537.36"}).text, "lxml")
    result = web_page.find_all(name="table", attrs={"class": "hrs-table"})
    if (len(result) == 0):
      break;
    imf_table = result[0]

    for i, row in enumerate(imf_table.tbody.find_all("tr")):
      # (i%4) = 0 data row
      if i % 4 == 0:
        tds = row.find_all("td")
        compte        = tds[1].text.replace ('\r\n ', '').replace('\n', '')
        nom           = tds[2].text.replace ('\r\n ', '').replace('\n', '')
        address       = tds[3].text.replace ('\r\n ', '').replace('\n', '')
        montant       = tds[4].text.replace ('\r\n ', '').replace('\n', '')
        beneficiare   = tds[5].text.replace ('\r\n ', '').replace('\n', '')
        bank = ""
        history = ""
    
      # (i%4) = 1 hidden row
      if i % 4 == 1:
        continue

      # (i%4) = 2 bank row
      if i % 4 == 2:
        tds = row.find_all("td")
        bank        = tds[0].text.replace ('\r\n ', '').replace('\n', '').replace ('Solde déclaré par: ','')
  
      # (i%4) = 3 history row
      if i % 4 == 3:
        tds = row.find_all("td")
        history        = tds[0].text.replace ('\r\n ', '').replace('\n', '')
        rows.append((compte, nom, address, montant, beneficiare, bank, history ))
  data_frame = pd.DataFrame(rows, columns=columns)
  data_frame.head()
  print(str (rows))

In [ ]:
  scrapePage ('John')

scrapePages
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=1&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=2&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=3&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=4&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=5&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=6&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=7&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=8&searchType=Person&lastName=John&propertyId=0
https://ubmswww.bank-banque-canada.ca/fr/Property/SearchIndex?page=9&searchT